In [1]:
# code replicated from https://github.com/charleywu/graphInference/blob/master/analyses/PXP.ipynb

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 

from bms import GroupBMC #https://github.com/cpilab/group-bayesian-model-comparison

# Protected exceedance probability across all conditions

## Controls

In [2]:
#Load data
m_control = pd.read_csv('modelResults/modelFits_control.csv', sep=',') #load data
m_control["model"] = m_control["kernel"] + "_" + m_control["acq"] #add model column

#Pivot table
df_wide = m_control.pivot_table(
    index="id",       # each row is one id
    columns="model",  # unique models become columns
    values="nLL"      # fill with nLL values
).reset_index()

selected_models = ["RBF_UCB", "BMT_UCB", "RBF_GM", "RBF_epsilonGreedy"]
df_filtered = df_wide[["id"] + selected_models]

In [3]:
#final numpy array to feed to BMS.py
m_control = -df_filtered[selected_models].to_numpy().T #transposed and nLLs made negative (i.e., log likelihoods from nLLs)

In [5]:
#compute pxp
result_control = GroupBMC(m_control).get_result().protected_exceedance_probability
result_control =  result_control.reshape(1, result_control.shape[0]) #reshape from column vector to row vector


In [6]:
result_control
# "RBF_UCB", "BMT_UCB", "RBF_GM", "RBF_epsilonGreedy"

array([[0.987688  , 0.00868264, 0.00192433, 0.00170503]])

## PD+

In [7]:
#Load data
m_PDplus = pd.read_csv('modelResults/modelFits_PD_plus.csv', sep=',') #load data
m_PDplus["model"] = m_PDplus["kernel"] + "_" + m_PDplus["acq"] #add model column

#Pivot table
df_wide = m_PDplus.pivot_table(
    index="id",       # each row is one id
    columns="model",  # unique models become columns
    values="nLL"      # fill with nLL values
).reset_index()

selected_models = ["RBF_UCB", "BMT_UCB", "RBF_GM", "RBF_epsilonGreedy"]
df_filtered = df_wide[["id"] + selected_models]

#final numpy array to feed to BMS.py
m_PDplus = -df_filtered[selected_models].to_numpy().T #transposed and nLLs made negative (i.e., log likelihoods from nLLs)

#compute pxp
result_PDplus = GroupBMC(m_PDplus).get_result().protected_exceedance_probability
result_PDplus =  result_PDplus.reshape(1, result_PDplus.shape[0]) #reshape from column vector to row vector


result_PDplus
# "RBF_UCB", "BMT_UCB", "RBF_GM", "RBF_epsilonGreedy"

array([[9.94803048e-01, 4.98925036e-03, 1.97182378e-04, 1.05188415e-05]])

## PD-

In [8]:
#Load data
m_PDminus = pd.read_csv('modelResults/modelFits_PD_minus.csv', sep=',') #load data
m_PDminus["model"] = m_PDminus["kernel"] + "_" + m_PDminus["acq"] #add model column

#Pivot table
df_wide = m_PDminus.pivot_table(
    index="id",       # each row is one id
    columns="model",  # unique models become columns
    values="nLL"      # fill with nLL values
).reset_index()

selected_models = ["RBF_UCB", "BMT_UCB", "RBF_GM", "RBF_epsilonGreedy"]
df_filtered = df_wide[["id"] + selected_models]

#final numpy array to feed to BMS.py
m_PDminus = -df_filtered[selected_models].to_numpy().T #transposed and nLLs made negative (i.e., log likelihoods from nLLs)

#compute pxp
result_PDminus = GroupBMC(m_PDminus).get_result().protected_exceedance_probability
result_PDminus =  result_PDminus.reshape(1, result_PDminus.shape[0]) #reshape from column vector to row vector


result_PDminus
# "RBF_UCB", "BMT_UCB", "RBF_GM", "RBF_epsilonGreedy"

array([[9.98073481e-01, 8.96649194e-04, 4.72173770e-04, 5.57695552e-04]])

# Put everything together and save

In [9]:
data = np.vstack([result_control, result_PDplus, result_PDminus])

# Build dataframe
df = pd.DataFrame(
    data,
    index=["control", "PD+", "PD-"],
    columns=["RBF_UCB", "BMT_UCB", "RBF_GM", "RBF_epsilonGreedy"]
)

In [10]:
df

,RBF_UCB,BMT_UCB,RBF_GM,RBF_epsilonGreedy
control,0.987688,0.008683,0.001924,0.001705
PD+,0.994803,0.004989,0.000197,0.000011
PD-,0.998073,0.000897,0.000472,0.000558


In [11]:
df.to_csv("modelResults/pxp.csv", index_label="condition")